# Solving 2D heat equation using PINN (PyTorch)

<img src="../images/2D_heat_eq.png" width="35%">

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import  seaborn as sns

## Define the PINN network

In [2]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

## Creating of data points (IC, BC)

In [3]:
def initial_condition(x, y):
    return torch.sin(torch.pi * x) * torch.sin(torch.pi * y)

def boundary_condition(x, y, t, custom_value):
    return torch.full_like(x, custom_value)

In [4]:
# generate collocation points
def generate_training_data(num_points):
    x = torch.rand(num_points, 1, requires_grad=True)  # torch.rand generate values interval [0, 1]
    y = torch.rand(num_points, 1, requires_grad=True)
    t = torch.rand(num_points, 1, requires_grad=True)
    return x, y, t

In [7]:
# generate boundary points
def generate_boundary_points(num_points):
    x_boundary = torch.tensor([0.0, 1.0]).repeat(num_points // 2) # generate BC for left and right side
    y_boundary = torch.rand(num_points) 

    if torch.rand(1) > 0.5:  # it can be changed to the bottom and up BC
        x_boundary, y_boundary = y_boundary, x_boundary
    
    return x_boundary.view(-1, 1), y_boundary.view(-1, 1)


def generate_boundary_training_data(num_points):
    x_boundary, y_boundary = generate_boundary_points(num_points)
    t = torch.rand(num_points, 1, requires_grad=True)  # ?

    return x_boundary, y_boundary, t

In [ ]:
def train_pinn(model, num_iterations, num_points):
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)

    for iteration in range(num_iterations):
        optimizer.zero_grad()

        x, y, t = generate_training_data(num_points)

        x_b, y_b, t_b = generate_boundary_training_data(num_points)

        t_initial = torch.zeros_like(t)
        u_initial = initial_condition(x, y)

        custom_value = 0
        u_boundary_x = boundary_condition(x_b, y_b, t_b, custom_value)
        u_boundary_y = boundary_condition(y_b, x_b, t_b, custom_value)
        

        



In [6]:
a = torch.tensor([0.0, 1.0]).repeat(10 // 2)
a

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])

In [ ]:
# def generate_boundary_points(num_points):
#     # Points on vertical boundaries x = 0 and x = 1
#     num_points_per_side = num_points // 4
    
#     x_left = torch.zeros(num_points_per_side)
#     y_left = torch.rand(num_points_per_side)
    
#     x_right = torch.ones(num_points_per_side)
#     y_right = torch.rand(num_points_per_side)
    
#     # Points on horizontal boundaries y = 0 and y = 1
#     y_bottom = torch.zeros(num_points_per_side)
#     x_bottom = torch.rand(num_points_per_side)
    
#     y_top = torch.ones(num_points_per_side)
#     x_top = torch.rand(num_points_per_side)
    
#     # Combine all boundary points
#     x_boundary = torch.cat((x_left, x_right, x_bottom, x_top))
#     y_boundary = torch.cat((y_left, y_right, y_bottom, y_top))
    
#     return x_boundary.view(-1, 1), y_boundary.view(-1, 1)

# # Example usage
# num_points = 100
# x_boundary, y_boundary = generate_boundary_points(num_points)
# print(x_boundary, y_boundary)